![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

In [1]:
from sqlalchemy import create_engine
import mysql.connector as mysql
import pandas as pd 


## Setting up the Connection String

In [2]:
# connection string parameters 
# set these values according to your mysql installation
user = 'root' 
password = 'Alberto1+'
#db = 'NUCLIO'
sqlServer = 'localhost' #should be IP or DNS of the db server

engine = create_engine("mysql+mysqlconnector://root:%s@localhost" % quote_plus(password))

# create connection string
connStr = "mysql+mysqlconnector://" + user + ":" + password + "@" + sqlServer#  +  "/"  + db

# print the connection string
print (connStr)

# we instantiate engine as a result of create_engine
conn = create_engine(connStr).connect() # connect to the server

NameError: name 'quote_plus' is not defined

## Executing SQL Queries in Python

In [5]:
# syntax: CREATE DATABASE dbName
sSQL = "CREATE DATABASE DAY2"
sSQLdrop= "DROP DATABASE IF EXISTS  DAY2"
conn.execute(sSQLdrop)
conn.execute(sSQL)

# other database creation and deletion commands
# conn.execute("DROP DATABASE DAY2") # to remove the db if already present, will return an error if missing
#conn.execute("DROP DATABASE IF EXISTS  DAY2") # to check whether the db exists before removing it


## Displaying the Execution Result

In [6]:
# command to see the dbs available in a db server
dbs=conn.execute("SHOW DATABASES")

See the number of rows in the result

In [7]:
dbs.rowcount

6

Display the rows

In [8]:
print(dbs.fetchall())

[('DAY2',), ('information_schema',), ('mysql',), ('NUCLIO',), ('performance_schema',), ('sys',)]


In [9]:
# command to specify the db to use
conn.execute("USE DAY2")


## DDL & DML Commands

In [10]:
# command to display available tables in the database
tables = conn.execute("SHOW TABLES")
tables.fetchall()

[]

## Creating a table from a file source

In [ ]:
pd.read_csv?

In [11]:
# using a csv file

# first read the csv data into a dataframe
products=pd.read_csv("files_sql/products.csv", sep=';', skipinitialspace=True)
products

,PRODUCT_ID,DESCRIPTION,PRICE
0,1,SAMSUNG S40,800
1,2,IPHONE 30,900
2,3,ONEPLUS 8T,750
3,4,PIXEL 10XL,800
4,5,HUAWEI P50 LITE,850


In [12]:
# then use the dataframe to populate the table
products.to_sql('products', if_exists='replace', con=conn, index=False)

In [13]:
# check that the table has been successfully added
conn.execute("SHOW TABLES").fetchall()

[('products',)]

In [14]:
# see the contents of the table
conn.execute("SELECT * FROM products").fetchall()


[(1, 'SAMSUNG S40', 800),
 (2, 'IPHONE 30', 900),
 (3, 'ONEPLUS 8T', 750),
 (4, 'PIXEL 10XL', 800),
 (5, 'HUAWEI P50 LITE', 850)]

In [27]:
# see the details of the table
# Documentation: https://dev.mysql.com/doc/refman/8.0/en/show-columns.html

conn.execute("SHOW COLUMNS FROM products").fetchall()

[('PRODUCT_ID', b'int', 'NO', 'PRI', None, ''),
 ('DESCRIPTION', b'text', 'YES', '', None, ''),
 ('PRICE', b'bigint', 'YES', '', None, '')]

In [26]:
# DDL Alter command
# Documentation: https://dev.mysql.com/doc/refman/8.0/en/alter-table.html

# an example to change the datatype of a column
conn.execute("ALTER TABLE products CHANGE COLUMN PRODUCT_ID PRODUCT_ID INT PRIMARY KEY")

In [19]:
# DML command to update the table
# Syntax: UPDATE tableName SET fieldName = newValue <WHERE> condition
conn.execute("UPDATE products SET description='ONEPLUS 5T' WHERE product_id=3")


In [20]:
conn.execute("SELECT * FROM products").fetchall()

[(1, 'SAMSUNG S40', 800),
 (2, 'IPHONE 30', 900),
 (3, 'ONEPLUS 5T', 750),
 (4, 'PIXEL 10XL', 800),
 (5, 'HUAWEI P50 LITE', 850)]

In [28]:
# using an xlsx file

# first read the excel data into a dataframe  
sellers=pd.read_excel("files_sql/Sellers.xlsx") #FULLY QUALIFIED NAME
sellers

,SELLER_ID,NAME,SURNAME,DOB,OFFICE_ID
0,1.0,CARLOS,PEREZ,1995-08-29,1.0
1,2.0,ANA,SUAREZ,1980-02-10,1.0
2,3.0,JUAN,SALAZAR,1990-05-08,1.0
3,4.0,PEDRO,GUTIERREZ,1980-01-07,2.0
4,5.0,MARIA,SOLIZ,2000-11-04,2.0


In [29]:
# add the table to mysql
sellers.to_sql('sellers', con=conn, if_exists='replace', index=False)

In [32]:
conn.execute("SELECT * FROM sellers").fetchall()

[(1.0, 'CARLOS', 'PEREZ', datetime.datetime(1995, 8, 29, 0, 0), 1.0),
 (2.0, 'ANA', 'SUAREZ', datetime.datetime(1980, 2, 10, 0, 0), 1.0),
 (3.0, 'JUAN', 'SALAZAR', datetime.datetime(1990, 5, 8, 0, 0), 1.0),
 (4.0, 'PEDRO', 'GUTIERREZ', datetime.datetime(1980, 1, 7, 0, 0), 2.0),
 (5.0, 'MARIA', 'SOLIZ', datetime.datetime(2000, 11, 4, 0, 0), 2.0),
 (6.0, 'MARTA', 'GALLARDO', datetime.datetime(1970, 1, 31, 0, 0), 2.0)]

In [31]:
# suppose we wish to add a seller with the following details
# seller_id: 6
# name: marta
# surname: gallardo
# dob: 31/01/1970
# office_id: 2
conn.execute("INSERT INTO sellers (SELLER_ID , NAME , SURNAME , DOB, OFFICE_ID) VALUES (6,'MARTA','GALLARDO','1970-01-31',2)")

In [33]:
# equivalent to SHOW COLUMNS FROM
conn.execute("DESCRIBE sellers").fetchall()

[('SELLER_ID', b'double', 'YES', '', None, ''),
 ('NAME', b'text', 'YES', '', None, ''),
 ('SURNAME', b'text', 'YES', '', None, ''),
 ('DOB', b'datetime', 'YES', '', None, ''),
 ('OFFICE_ID', b'double', 'YES', '', None, '')]

In [36]:
# suppose we wish to delete a seller that has left the company

# DML Command to delete FROM the table
# Note the difference that a DROP command (DDL) removes the whole entity/table from the db, 
# but the DELETE command (DML) only removes the contents of the entity i.e. the data/row within the table

# Syntax: DELETE FROM tableName <WHERE> condition
conn.execute("DELETE FROM sellers WHERE seller_ID=4")


In [40]:
# see table contents
conn.execute("SELECT * FROM sellers").fetchall() #MySQL is not case senstive

[(1.0, 'CARLOS', 'PEREZ', datetime.datetime(1995, 8, 29, 0, 0), 1.0),
 (2.0, 'ANA', 'SUAREZ', datetime.datetime(1980, 2, 10, 0, 0), 1.0),
 (3.0, 'JUAN', 'SALAZAR', datetime.datetime(1990, 5, 8, 0, 0), 1.0),
 (5.0, 'MARIA', 'SOLIZ', datetime.datetime(2000, 11, 4, 0, 0), 2.0),
 (6.0, 'MARTA', 'GALLARDO', datetime.datetime(1970, 1, 31, 0, 0), 2.0)]

In [41]:
# using an html file

# first read the html data into a dataframe
clients=pd.read_html("files_sql/clients.html")
clients[0]

,CLIENT_ID,NAME,SURNAME
0,1,FRED,BARNES
1,2,KELLY,JONES
2,3,SANDRA,LOPEZ


In [42]:
# add the table to mysql
clients[0].to_sql('clients', con=conn, if_exists='replace', index=False)
conn.execute("SELECT * FROM clients").fetchall()

[(1, 'FRED', 'BARNES'), (2, 'KELLY', 'JONES'), (3, 'SANDRA', 'LOPEZ')]

In [43]:
# import remaining data from files
sales_jan=pd.read_csv("files_sql/sales_jan.csv",skipinitialspace=True, sep=";")
sales_jan

,CLIENT_ID,VENDOR_ID,PRODUCT_ID,DATE_OF_SALE,SALE_ID
0,1,1,1,17/01/2020,1
1,3,1,2,19/01/2020,2
2,2,1,3,09/01/2020,3
3,3,3,4,07/01/2020,4
4,2,3,5,20/01/2020,5
5,2,2,5,08/01/2020,6
6,2,2,5,04/01/2020,7
7,2,2,5,14/01/2020,8
8,1,5,5,25/01/2020,9
9,1,6,4,14/01/2020,10


In [44]:
type(sales_jan["DATE_OF_SALE"][0])

str

In [45]:
# we may wish to update datatypes before we write to the db
sales_jan["DATE_OF_SALE"] = pd.to_datetime(sales_jan["DATE_OF_SALE"],format="%d/%m/%Y") #YYYY-MM-DD, 

In [46]:
type(sales_jan["DATE_OF_SALE"][0])

pandas._libs.tslibs.timestamps.Timestamp

In [47]:
sales_jan["DATE_OF_SALE"]

0    2020-01-17
1    2020-01-19
2    2020-01-09
3    2020-01-07
4    2020-01-20
5    2020-01-08
6    2020-01-04
7    2020-01-14
8    2020-01-25
9    2020-01-14
10   2020-01-14
11   2020-01-21
12   2020-01-22
13   2020-01-13
14   2020-01-10
15   2020-01-25
16   2020-01-25
17   2020-01-04
18   2020-01-18
19   2020-01-06
20   2020-01-05
21   2020-01-19
22   2020-01-11
23   2020-01-22
24   2020-01-11
25   2020-01-04
26   2020-01-02
27   2020-01-05
28   2020-01-13
Name: DATE_OF_SALE, dtype: datetime64[ns]

In [48]:
sales_feb = pd.read_csv("files_sql/sales_feb.csv",skipinitialspace=True, sep=";")
sales_feb["DATE_OF_SALE"] = pd.to_datetime(sales_feb["DATE_OF_SALE"],format="%d/%m/%Y")

In [52]:
sales_jan.head()

,CLIENT_ID,VENDOR_ID,PRODUCT_ID,DATE_OF_SALE,SALE_ID
0,1,1,1,2020-01-17,1
1,3,1,2,2020-01-19,2
2,2,1,3,2020-01-09,3
3,3,3,4,2020-01-07,4
4,2,3,5,2020-01-20,5


In [51]:
sales_feb.head()

,CLIENT_ID,VENDOR_ID,PRODUCT_ID,DATE_OF_SALE,SALE_ID
0,3,2,5,2020-02-06,30
1,3,3,3,2020-02-14,31
2,2,2,5,2020-02-20,32
3,3,2,4,2020-02-15,33
4,2,2,3,2020-02-08,34


### How can we import sales from january and february into a single table?

In [58]:
sales = pd.concat([sales_jan, sales_feb])
sales.head()

,CLIENT_ID,VENDOR_ID,PRODUCT_ID,DATE_OF_SALE,SALE_ID
0,1,1,1,2020-01-17,1
1,3,1,2,2020-01-19,2
2,2,1,3,2020-01-09,3
3,3,3,4,2020-01-07,4
4,2,3,5,2020-01-20,5


In [59]:
sales_final = sales.dropna(axis='index', how='all')

In [64]:
conn.execute("DROP TABLE sales")

In [65]:
sales_final.to_sql("sales", con=conn, if_exists="replace", index=False) #take a look why replace doesnt work if table exists

In [66]:
conn.execute("SHOW COLUMNS FROM sales").fetchall()

[('CLIENT_ID', b'bigint', 'YES', '', None, ''),
 ('VENDOR_ID', b'bigint', 'YES', '', None, ''),
 ('PRODUCT_ID', b'bigint', 'YES', '', None, ''),
 ('DATE_OF_SALE', b'datetime', 'YES', '', None, ''),
 ('SALE_ID', b'bigint', 'YES', '', None, '')]

In [ ]:
sales = pd.concat([sales_jan,sales_feb])
sales

In [ ]:
# it is good to always check for any null rows unfamiliar datasets
sales_final = sales.dropna(axis='index',how='all')

In [ ]:
# write to mysql
sales_final.to_sql("sales", con=conn, index=False, if_exists='replace')
conn.execute("DESC sales").fetchall()

In [68]:
conn.execute("DROP TABLE sales")

In [69]:
# another way to do this is by combining an insert and select

#create sales from sales_jan dataframe
sales_jan_final = sales_jan.dropna(axis=0, how='all') 
sales_jan_final.to_sql("sales", con=conn, index=False, if_exists='replace')

#create sales_trmp from sales_feb dataframe
sales_feb_final = sales_feb.dropna(axis=0, how='all')
sales_feb_final.to_sql("sales_temp", con=conn, index=False, if_exists='replace')

# insert into sales the data from sales_temp
conn.execute("INSERT INTO sales (CLIENT_ID, VENDOR_ID, PRODUCT_ID, DATE_OF_SALE, SALE_ID) \
                        SELECT CLIENT_ID, VENDOR_ID, PRODUCT_ID, DATE_OF_SALE, SALE_ID FROM sales_temp")

conn.execute("SELECT * FROM sales").fetchall()

conn.execute("DROP TABLE sales_temp")


In [70]:
conn.execute("SELECT * FROM sales").fetchall()

[(1, 1, 1, datetime.datetime(2020, 1, 17, 0, 0), 1),
 (3, 1, 2, datetime.datetime(2020, 1, 19, 0, 0), 2),
 (2, 1, 3, datetime.datetime(2020, 1, 9, 0, 0), 3),
 (3, 3, 4, datetime.datetime(2020, 1, 7, 0, 0), 4),
 (2, 3, 5, datetime.datetime(2020, 1, 20, 0, 0), 5),
 (2, 2, 5, datetime.datetime(2020, 1, 8, 0, 0), 6),
 (2, 2, 5, datetime.datetime(2020, 1, 4, 0, 0), 7),
 (2, 2, 5, datetime.datetime(2020, 1, 14, 0, 0), 8),
 (1, 5, 5, datetime.datetime(2020, 1, 25, 0, 0), 9),
 (1, 6, 4, datetime.datetime(2020, 1, 14, 0, 0), 10),
 (1, 6, 3, datetime.datetime(2020, 1, 14, 0, 0), 11),
 (1, 6, 3, datetime.datetime(2020, 1, 21, 0, 0), 12),
 (1, 6, 3, datetime.datetime(2020, 1, 22, 0, 0), 13),
 (2, 5, 2, datetime.datetime(2020, 1, 13, 0, 0), 14),
 (3, 5, 2, datetime.datetime(2020, 1, 10, 0, 0), 15),
 (3, 5, 2, datetime.datetime(2020, 1, 25, 0, 0), 16),
 (3, 5, 2, datetime.datetime(2020, 1, 25, 0, 0), 17),
 (3, 5, 2, datetime.datetime(2020, 1, 4, 0, 0), 18),
 (3, 3, 1, datetime.datetime(2020, 1, 18, 

## DQL - SELECT Command

In [ ]:
# Syntax of the SELECT statement

# Note that WHERE, GROUP BY, HAVING and ORDER BY are all optional clauses

# Optional clauses are often indicated with square brackets in SQL documentation

# Using * instead of field names will select all fields

sSQL = """
    SELECT
        field1, field2
    FROM
        tableName
    [WHERE]
        condition1
    [GROUP BY]
        fieldList
    [HAVING]
        condition1
    [ORDER BY]
        fieldList [DESC]
    [LIMIT]
        integer
"""

### Order of Operations:

In [ ]:
# FROM - TABLE(s) to read FROM
# WHERE - Filters the data
# GROUP BY - Aggregates the data
# HAVING - Filters the aggregated data
# SELECT - Returns the final data
# ORDER BY - Orders the final data
# LIMIT - Returns the number of rows specified

In [ ]:
sSQL = """
    /*
        Comment that spans 
        multiple lines
        part of my comment
        
    */
    # Comment that spans a single line
    SELECT 
        lastName, firstName -- Comment to the end of the line
    FROM
        employees
"""

In [71]:
sSQLshort = "SELECT * FROM sales LIMIT 5"

sSQL = """
    SELECT *
    FROM sales 
    LIMIT 5   
"""

conn.execute(sSQL).fetchall()

[(1, 1, 1, datetime.datetime(2020, 1, 17, 0, 0), 1),
 (3, 1, 2, datetime.datetime(2020, 1, 19, 0, 0), 2),
 (2, 1, 3, datetime.datetime(2020, 1, 9, 0, 0), 3),
 (3, 3, 4, datetime.datetime(2020, 1, 7, 0, 0), 4),
 (2, 3, 5, datetime.datetime(2020, 1, 20, 0, 0), 5)]

In [72]:
# setting a primary key

sSQL = """
    CREATE TABLE Offices (
        office_id INT PRIMARY KEY,
        name VARCHAR(50)
    )
"""
conn.execute(sSQL)

In [73]:
# inserting into the table
conn.execute("INSERT INTO Offices(office_id, name) VALUES(1,'BARCELONA')")
conn.execute("INSERT INTO Offices(office_id, name) VALUES(2,'MADRID')")
conn.execute("INSERT INTO Offices(office_id, name) VALUES(3,'VALENCIA')")

results = conn.execute("SELECT * FROM Offices")
print(results.fetchall())

[(1, 'BARCELONA'), (2, 'MADRID'), (3, 'VALENCIA')]


In [74]:
conn.execute("SHOW TABLES").fetchall()

[('clients',), ('Offices',), ('products',), ('sales',), ('sellers',)]

In [79]:
conn.execute("SHOW COLUMNS FROM OFFICES").fetchall()

[('office_id', b'int', 'NO', 'PRI', None, ''),
 ('name', b'varchar(50)', 'YES', '', None, '')]

In [80]:
# WARM-UP EXERCISE 1
# Select client ID and vendor ID
# Using the ORDER BY clause to sort results by descending vendor ID
# and ascending client ID
sSQL = """
    SELECT 
        CLIENT_ID,
        VENDOR_ID
    FROM sales
    ORDER BY VENDOR_ID DESC, CLIENT_ID ASC 
""" 

pd.read_sql(sSQL,con=conn)

,CLIENT_ID,VENDOR_ID
0,1,6
1,1,6
2,1,6
3,1,6
4,1,6
5,1,6
6,2,6
7,3,6
8,3,6
9,3,6


In [81]:
# WARM-UP EXERCISE 2

# Select the maximum sale ID in each month and display
# the result if the sale ID is an even number

# Documentation on date-time functions: https://dev.mysql.com/doc/refman/8.0/en/date-and-time-functions.html


sSQL = """
    SELECT
        MAX(SALE_ID),
        MONTH(DATE_OF_SALE)
    FROM sales
    WHERE SALE_ID % 2 = 0
    GROUP BY MONTH(DATE_OF_SALE)
"""
pd.read_sql(sSQL,con=conn)

,MAX(SALE_ID),MONTH(DATE_OF_SALE)
0,28,1
1,58,2


In [82]:
sSQL = """
    SELECT
        MAX(SALE_ID),
        MONTH(DATE_OF_SALE)
    FROM sales
    GROUP BY MONTH(DATE_OF_SALE)
    HAVING (MAX(SALE_ID) % 2 = 0)
"""
pd.read_sql(sSQL,con=conn)

,MAX(SALE_ID),MONTH(DATE_OF_SALE)
0,58,2


In [83]:
#Check if there was a sale_id greater than 28 for January
sSQL = """
    SELECT
       SALE_ID
    FROM sales
    WHERE MONTH(DATE_OF_SALE) = 1
    AND SALE_ID > 28
"""

pd.read_sql(sSQL,con=conn)

,SALE_ID
0,29


In [86]:
sales

,CLIENT_ID,VENDOR_ID,PRODUCT_ID,DATE_OF_SALE,SALE_ID
0,1,1,1,2020-01-17,1
1,3,1,2,2020-01-19,2
2,2,1,3,2020-01-09,3
3,3,3,4,2020-01-07,4
4,2,3,5,2020-01-20,5
5,2,2,5,2020-01-08,6
6,2,2,5,2020-01-04,7
7,2,2,5,2020-01-14,8
8,1,5,5,2020-01-25,9
9,1,6,4,2020-01-14,10


#### Finally! Lets work through the practical example

In [85]:
# 1. Quantity of products sold - requires counting -- COUNT()
sSQL = """
    SELECT
        COUNT(*) AS 'Total'
    FROM sales
"""
pd.read_sql(sSQL,con=conn)


,Total
0,58


In [88]:
# 2. List of products with prices higher than 700 euros
sSQL = """
    SELECT description, price 
    FROM products
    WHERE price > 800
"""
pd.read_sql(sSQL,con=conn)


,description,price
0,IPHONE 30,900
1,HUAWEI P50 LITE,850


In [90]:
# 3. List of vendors under the age of 30
sSQL = """
    SELECT *
    FROM sellers
    WHERE 
        dob >= '1993-01-01'
"""
pd.read_sql(sSQL, con=conn)

,SELLER_ID,NAME,SURNAME,DOB,OFFICE_ID
0,1.0,CARLOS,PEREZ,1995-08-29,1.0
1,5.0,MARIA,SOLIZ,2000-11-04,2.0


In [99]:
# An alternate way to calculate the age of each seller?
# Hint! Use the TIMESTAMPDIFF function
sSQL = """
    SELECT 
        NAME,
        SURNAME,
        TIMESTAMPDIFF(YEAR, DOB, NOW()) AS AGE
    FROM sellers
    WHERE TIMESTAMPDIFF(YEAR, DOB, NOW()) < 30
"""
pd.read_sql(sSQL, con=conn)

,NAME,SURNAME,AGE
0,CARLOS,PEREZ,28
1,MARIA,SOLIZ,23


In [100]:
# 4. List of vendors with province with name and surname
sSQL = """
    SELECT
        s.name,
        s.surname,
        o.name AS province
    FROM sellers AS s LEFT JOIN offices AS o
        ON s.OFFICE_ID = o.OFFICE_ID
"""
pd.read_sql(sSQL, con=conn)

,name,surname,province
0,CARLOS,PEREZ,BARCELONA
1,ANA,SUAREZ,BARCELONA
2,JUAN,SALAZAR,BARCELONA
3,MARIA,SOLIZ,MADRID
4,MARTA,GALLARDO,MADRID


In [ ]:
# 5. List of offices with their assigned vendors (Consider all offices)
sSQL = """

"""
pd.read_sql(sSQL,con=conn)


In [ ]:
# 5. List of vendors with their assigned offices
sSQL = """

"""
pd.read_sql(sSQL,con=conn)

In [106]:
# 6. List of offices that do not have vendors assigned
# (Consider all offices)

#sellers that do not have an office id associated
sSQL2 = """
    SELECT *
    FROM sellers AS s RIGHT JOIN offices AS o
        ON s.OFFICE_ID = o.OFFICE_ID
    WHERE s.OFFICE_ID IS NULL
"""
#offices that do not have sellers with an office id associated
sSQL = """
    SELECT *
    FROM offices AS o LEFT JOIN sellers AS s
        ON s.OFFICE_ID = o.OFFICE_ID
    WHERE s.OFFICE_ID IS NULL
"""

pd.read_sql(sSQL2,con=conn)

,SELLER_ID,NAME,SURNAME,DOB,OFFICE_ID,office_id,name
0,None,None,None,None,None,3,VALENCIA


In [114]:
# 7. List of offices with the quantity sold per seller
# (Consider all offices)
sSQL = """
    SELECT
        o.OFFICE_ID,
        o.name,
        s.SELLER_ID,
        s.NAME,
        s.SURNAME,
        COUNT(VENDOR_ID) AS Quantity_Sales
    FROM 
        offices AS o LEFT JOIN sellers AS s
            ON o.OFFICE_ID = s.OFFICE_ID
        RIGHT JOIN SALES AS sa
            ON sa.VENDOR_ID = s.SELLER_ID
    GROUP BY o.OFFICE_ID, o.name, s.SELLER_ID, s.NAME, s.SURNAME
    HAVING COUNT(VENDOR_ID) > 10
    ORDER BY s.name
"""
pd.read_sql(sSQL,con=conn)



,OFFICE_ID,name,SELLER_ID,NAME,SURNAME,Quantity_Sales
0,1,BARCELONA,2.0,ANA,SUAREZ,20
1,1,BARCELONA,3.0,JUAN,SALAZAR,14


In [ ]:
# 8. Income from sales between January 15 to February 15

sSQL = """

"""
pd.read_sql(sSQL,con=conn)


In [ ]:
sSQL = """
"""

pd.read_sql(sSQL,con=conn)

In [ ]:
# 9. List of products sold (With their description, price, quantity sold and income)
sSQL = """
"""

pd.read_sql(sSQL,con=conn)


In [ ]:
# Alternate Solution using a CROSS PRODUCT (sometimes called a CROSS JOIN)
# Note that this approach is much less efficient than the previous
# When we do not specify a JOIN keyword, the cross product of the tables specified in the FROM clause is generated
sSQL = """

"""
pd.read_sql(sSQL, con=conn)

In [ ]:
sSQL = """
"""
pd.read_sql(sSQL, con=conn)

In [ ]:
# 10. Ranking of best sellers/vendors (criterion: by quantity of products sold)
sSQL = """
"""
pd.read_sql(sSQL, con=conn)

In [ ]:
# 11. Ranking of the best clients (criterion: by income obtained by the company)
# those who have purchased the most
# The sql function CONCAT() can be used to concatenate text fields
sSQL = """
"""
pd.read_sql(sSQL, con=conn)

In [ ]:
# 12. Ranking of the best clients who have the Barcelona office as a seller (criterion: by income obtained)
sSQL = """
"""

pd.read_sql(sSQL, con=conn)

In [ ]:
# 13. Count of products that have more than 10 products sold
sSQL = """
"""
pd.read_sql(sSQL, con=conn)



In [ ]:
# 14. A final sales report with the following format: 
# First and last name of the seller, 
# office of the seller, 
# first and last name of the customer,
# name of the product and price, 
# date of sale
sSQL = """
"""

pd.read_sql(sSQL, con=conn)

In [ ]:
# When we have generated a useful query like the above, SQL makes it possible to reuse the query by nesting it inside another
# This is called making use of an Inner Query

sSQL = """
"""
pd.read_sql(sSQL, con=conn)

In [ ]:
# closing the connection to the db
conn.close()